In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap

from scipy.stats import binom

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:
omdf = pd.read_excel(rootdir_data+'Omikron.xlsx')
omdf['Dato'] = pd.to_datetime(omdf['Dato'])
omdf['Ratio'] = omdf['AntalOmikron']/omdf['AntalTest']
omdf['Perc'] = 100 * omdf['AntalOmikron']/omdf['AntalTest']


In [3]:
# Calculate error of data, assuming binomial distribution
mean, var, skew, kurt = binom.stats(omdf.AntalTest,omdf.Ratio,moments = 'mvsk')
omdf['mean'] = mean 
omdf['var'] = var 
omdf['err'] = np.sqrt(var)/omdf.AntalTest
display(omdf)


,Dato,AntalTest,AntalOmikron,Ratio,Perc,mean,var,err
0,2021-11-27,3981,3,0.000754,0.075358,3.0,2.997739,0.000435
1,2021-11-28,3921,11,0.002805,0.280541,11.0,10.969141,0.000845
2,2021-11-29,4959,12,0.002420,0.241984,12.0,11.970962,0.000698
3,2021-11-30,5315,24,0.004516,0.451552,24.0,23.891627,0.000920
4,2021-12-01,4404,77,0.017484,1.748411,77.0,75.653724,0.001975
5,2021-12-02,4449,63,0.014160,1.416049,63.0,62.107889,0.001771
6,2021-12-03,5087,77,0.015137,1.513662,77.0,75.834480,0.001712
7,2021-12-04,5166,112,0.021680,2.168022,112.0,109.571816,0.002026
8,2021-12-05,5066,169,0.033360,3.335965,169.0,163.362219,0.002523
9,2021-12-06,6961,355,0.050998,5.099842,355.0,336.895561,0.002637


In [4]:

firstDate = np.datetime64('2021-10-01')

latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir
latestdir

dfCase = pd.read_csv(latestdir+'\\Test_pos_over_time.csv',dtype='str',delimiter=';',encoding='latin1')

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')
# dfAge['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(dfAge['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

# dfAge.tail(18)
dfCase = dfCase.iloc[:-2] 
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].str.replace('.',''))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].str.replace('.',''))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].str.replace(',','.'))
dfCase['Date'] = pd.to_datetime(dfCase['Date'])
dfCase.tail()

<ipython-input-4-b96e666e5382>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].str.replace('.',''))
<ipython-input-4-b96e666e5382>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].str.replace('.',''))


,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
684,2021-12-11,6602,143.721,4.6,10.253,153974,49.750.916
685,2021-12-12,7015,153.700,4.6,11.699,165399,49.916.315
686,2021-12-13,10334,212.381,4.9,16.924,229305,50.145.620
687,2021-12-14,10016,186.125,5.4,14.802,200927,50.346.547
688,2021-12-15,829,20.679,4.0,1.899,22578,50.369.125


In [5]:
# Plot general case data
fig,ax1 = plt.subplots(tight_layout=True)

ax1.fill_between(dfCase.Date,dfCase.NewPositive,color='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:

omdfShort = omdf.iloc[:-1]
omdfShort

firstOmikronDate = omdfShort.Dato.iloc[0]
lastOmikronDate = omdfShort.Dato.iloc[-1]
# np.where(dfCase.Date == firstOmikronDate)[0][0]
df_withOm = dfCase[(dfCase.Date >= firstOmikronDate) & (dfCase.Date <= lastOmikronDate)]
df_withOm.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
683,2021-12-10,6831,196.222,3.5,14.481,210703,49.596.942
684,2021-12-11,6602,143.721,4.6,10.253,153974,49.750.916
685,2021-12-12,7015,153.700,4.6,11.699,165399,49.916.315
686,2021-12-13,10334,212.381,4.9,16.924,229305,50.145.620
687,2021-12-14,10016,186.125,5.4,14.802,200927,50.346.547


In [7]:
# omdf.tail()
omdfShort

,Dato,AntalTest,AntalOmikron,Ratio,Perc,mean,var,err
0,2021-11-27,3981,3,0.000754,0.075358,3.0,2.997739,0.000435
1,2021-11-28,3921,11,0.002805,0.280541,11.0,10.969141,0.000845
2,2021-11-29,4959,12,0.002420,0.241984,12.0,11.970962,0.000698
3,2021-11-30,5315,24,0.004516,0.451552,24.0,23.891627,0.000920
4,2021-12-01,4404,77,0.017484,1.748411,77.0,75.653724,0.001975
5,2021-12-02,4449,63,0.014160,1.416049,63.0,62.107889,0.001771
6,2021-12-03,5087,77,0.015137,1.513662,77.0,75.834480,0.001712
7,2021-12-04,5166,112,0.021680,2.168022,112.0,109.571816,0.002026
8,2021-12-05,5066,169,0.033360,3.335965,169.0,163.362219,0.002523
9,2021-12-06,6961,355,0.050998,5.099842,355.0,336.895561,0.002637


In [8]:
# fig,ax1 = plt.subplots(tight_layout=True)
fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)
ax1.fill_between(dfCase.Date,dfCase.NewPositive,color='gray')

# ax1.plot(df_withOm.Date,df_withOm.NewPositive.values - omdf.AntalOmikron)
ax1.fill_between(df_withOm.Date,df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values,color='r')
# ax1.fill_between(omdf.Dato,omdf.AntalOmikron,color='r')
# ax1.fill_between(omdf.Dato,omdf.AntalOmikron,color='b')

ax2.fill_between(dfCase.Date,100*np.ones(dfCase.Date.shape),color='gray')
ax2.fill_between(df_withOm.Date,100*(1-np.divide(df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values)),color='r')


ax1.set_xlim(left=np.datetime64('2021-11-10'),right=omdfShort.Dato.max())
ax2.set_ylim([0,100])
# omdf
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig,ax1 = plt.subplots(tight_layout=True)

ax1.plot(dfCase.Date,dfCase.NewPositive,'.:',color='k',linewidth=0.5)
ax1.fill_between(rnTime(dfCase.Date.values,7),rnMean(dfCase.NewPositive.values,7),color='gray')

ax1.fill_between(rnTime(df_withOm.Date.values,7),rnMean(df_withOm.NewPositive.values - omdfShort.AntalOmikron,7),rnMean(df_withOm.NewPositive.values,7),color='r')

ax1.set_xlim(left=np.datetime64('2021-10-10'),right=omdfShort.Dato.max()-np.timedelta64(3,'D'))

ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)

ax1.plot(dfCase.Date,dfCase.NewPositive,'.:',color='k',linewidth=0.5)
ax1.fill_between(rnTime(dfCase.Date.values,7),rnMean(dfCase.NewPositive.values,7),color='gray')

ax1.fill_between(rnTime(df_withOm.Date.values,7),rnMean(df_withOm.NewPositive.values - omdfShort.AntalOmikron,7),rnMean(df_withOm.NewPositive.values,7),color='r')

ax1.set_xlim(left=np.datetime64('2021-10-10'),right=omdfShort.Dato.max()-np.timedelta64(3,'D'))

ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

ax2.fill_between(dfCase.Date,100*np.ones(dfCase.Date.shape),color='gray')
ax2.fill_between(df_withOm.Date,100*(1-np.divide(df_withOm.NewPositive.values - omdfShort.AntalOmikron,df_withOm.NewPositive.values)),color='r')


ax1.set_xlim(left=np.datetime64('2021-11-10'),right=omdfShort.Dato.max())
ax2.set_ylim([0,100])
# omdf
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …